## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd

In [4]:
df=pd.read_csv('train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
###Drop Nan Values
df=df.dropna()

In [7]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
y.value_counts()

,count
label,
0,10361
1,7924


In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [15]:
### Vocabulary size
voc_size=5000

### Vector Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords
import numpy as np

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
messages = X.copy()
messages.reset_index(inplace=True)

In [41]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [44]:
from tensorflow.keras.preprocessing.text import one_hot

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[327, 2485, 4363, 2582, 3604, 3769, 1805, 4807, 3545, 1097],
 [1161, 635, 834, 4916, 2275, 2450, 282],
 [2851, 1363, 642, 1567],
 [3955, 2070, 1535, 4385, 1869, 2273],
 [1493, 2275, 4621, 2825, 4901, 2094, 2275, 3440, 2096, 4988],
 [663,
  3809,
  1267,
  1217,
  2325,
  4014,
  4022,
  4264,
  3187,
  528,
  3622,
  4668,
  691,
  1799,
  282],
 [3601, 1659, 514, 1365, 2686, 4798, 1718, 3569, 4834, 821, 1210],
 [1867, 4132, 3420, 716, 93, 4059, 4014, 3704, 4834, 821, 1210],
 [2249, 3210, 377, 1197, 2201, 1475, 563, 4842, 4014, 1472],
 [4503, 3554, 764, 2535, 4919, 1139, 2077, 3600],
 [2519, 3546, 899, 1181, 3123, 4380, 4769, 4800, 4716, 1581, 3269],
 [4385, 2166, 3604, 1475, 4014, 93],
 [4584, 1422, 2762, 1144, 3721, 493, 4791, 3861, 2250],
 [529, 49, 1375, 4085, 3352, 3566, 4062, 4834, 821, 1210],
 [2508, 1820, 1751, 4028, 2139, 4834, 821, 1210],
 [260, 1046, 2165, 3164, 3177, 3239, 4882, 758, 4352, 2555],
 [2215, 4610, 635],
 [2774, 4112, 4446, 2161, 4014, 4499, 1768, 282],
 [3378,

In [46]:
sent_length=20
embedded_sent=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_sent)

[[   0    0    0 ... 4807 3545 1097]
 [   0    0    0 ... 2275 2450  282]
 [   0    0    0 ... 1363  642 1567]
 ...
 [   0    0    0 ... 4834  821 1210]
 [   0    0    0 ... 4337 1310 3552]
 [   0    0    0 ... 3289 1315 4588]]


In [48]:
## Creating LSTM model
dim = 40
model_LSTM=Sequential()
model_LSTM.add(Embedding(voc_size,dim,input_length=sent_length))
model_LSTM.add(LSTM(100))
model_LSTM.add(Dense(1,activation='sigmoid'))
model_LSTM.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [50]:
## Creating Bi-directional LSTM model
model_Bi=Sequential()
model_Bi.add(Embedding(voc_size,dim,input_length=sent_length))
model_Bi.add(Bidirectional(LSTM(100)))
model_Bi.add(Dropout(0.3))
model_Bi.add(Dense(1,activation='sigmoid'))
model_Bi.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [51]:
len(embedded_sent),y.shape

(18285, (18285,))

In [52]:
import numpy as np
X_new=np.array(embedded_sent)
y_new=np.array(y)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.3, random_state=42)

### Model Training

In [54]:
model_LSTM.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.7965 - loss: 0.4198 - val_accuracy: 0.9136 - val_loss: 0.2033
Epoch 2/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9437 - loss: 0.1476 - val_accuracy: 0.9189 - val_loss: 0.1950
Epoch 3/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9608 - loss: 0.1022 - val_accuracy: 0.9167 - val_loss: 0.2058
Epoch 4/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9780 - loss: 0.0695 - val_accuracy: 0.9167 - val_loss: 0.2302
Epoch 5/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9801 - loss: 0.0588 - val_accuracy: 0.9176 - val_loss: 0.3093
Epoch 6/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9877 - loss: 0.0422 - val_accuracy: 0.9140 - val_loss: 0.3280
Epoch 7/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9917 - loss: 0.0284 - val_accuracy: 0.9105 - val_loss: 0.3685
Epoch 8/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9928 - loss: 0.0254 - val_accuracy: 0

In [55]:
### Finally Training
model_Bi.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.7889 - loss: 0.4081 - val_accuracy: 0.9192 - val_loss: 0.1947
Epoch 2/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9508 - loss: 0.1291 - val_accuracy: 0.9171 - val_loss: 0.2035
Epoch 3/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9659 - loss: 0.0938 - val_accuracy: 0.9127 - val_loss: 0.2466
Epoch 4/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9751 - loss: 0.0716 - val_accuracy: 0.9120 - val_loss: 0.2434
Epoch 5/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9826 - loss: 0.0584 - val_accuracy: 0.9147 - val_loss: 0.3330
Epoch 6/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9881 - loss: 0.0402 - val_accuracy: 0.9121 - val_loss: 0.3361
Epoch 7/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9898 - loss: 0.0307 - val_accuracy: 0.9103 - val_loss: 0.3802
Epoch 8/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9913 - loss: 0.0291 - val_accuracy:

### Performance Metrics And Accuracy

In [67]:
y_pred1=model_LSTM.predict(X_test)
y_pred1 = np.argmax(y_pred1, axis=1)

172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [68]:
y_pred2=model_Bi.predict(X_test)
y_pred2 = np.argmax(y_pred2, axis=1)

172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [69]:
from sklearn.metrics import confusion_matrix

In [72]:
confusion_matrix(y_test,y_pred1)

array([[3107,    0],
       [2379,    0]])

In [73]:
confusion_matrix(y_test,y_pred2)

array([[3107,    0],
       [2379,    0]])

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.566350710900474

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred2)

0.566350710900474

In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3107
           1       0.00      0.00      0.00      2379

    accuracy                           0.57      5486
   macro avg       0.28      0.50      0.36      5486
weighted avg       0.32      0.57      0.41      5486



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3107
           1       0.00      0.00      0.00      2379

    accuracy                           0.57      5486
   macro avg       0.28      0.50      0.36      5486
weighted avg       0.32      0.57      0.41      5486



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
